In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from ipfn import ipfn

/Users/ritarodriguez/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Data 2019

## Aggregated 2019

We have the following poer indicators, aggregated for the whole year 2019: 
http://epsis.kpx.or.kr/epsisnew/selectEkesKepChart.do?menuId=010400


In [2]:
# Power indicator
df_2019 = pd.read_csv('HOME_MajorIndicators_PowerIndicators.csv')

df_2019

,Unnamed: 0,Hydro Power(MW),Thermal power(MW),Nuclear (MW),Car (MW),Total (MW),Total hydraulic energy(GWh),total fire energy(GWh),total nuclear energy(GWh),total car energy(GWh),Total energy (Gwh)
0,2019,6508.103,80478.222,23250,3753.91,129091.579,6247.064,377844.247,145909.669,23407.545,585301.444


## By month and by fuel 2019
We also have generation capacity information broken down by month and fuel: http://epsis.kpx.or.kr/epsisnew/selectEkpoBftChart.do?menuId=020100&locale=eng
Unit: MW

In [3]:
df_by_fuel = pd.read_csv('HOME_GenerationCapacity_byFuel.csv')
df_by_fuel = df_by_fuel.rename(columns={"New&Renewable energy": "Fuel Cell", "Unnamed: 9": "IGCC",
"Unnamed: 10": "Solar Power",
"Unnamed: 11": "Wind Power",
"Unnamed: 12": "Hydro Power",
"Unnamed: 13": "Marine Energy",
"Unnamed: 14": "Bio Energy",
"Unnamed: 15": "Waste Energy"
})
df_by_fuel = df_by_fuel.drop([0]).reset_index(drop=True)

df_by_fuel['Period'] = pd.to_datetime(df_by_fuel['Period'])

cols = ["Fuel Cell", "IGCC", "Solar Power", "Wind Power",
        "Hydro Power", "Marine Energy", "Bio Energy", "Waste Energy"]
df_by_fuel[cols] = df_by_fuel[cols].apply(pd.to_numeric, errors='coerce', axis=1)


In [4]:
df_by_fuel_2019 = df_by_fuel[df_by_fuel['Period'].dt.year == 2019].reset_index(drop=True).copy()

#####
df_by_fuel_2019['Num Days'] = df_by_fuel_2019['Period'].dt.days_in_month
df_by_fuel_2019['Month'] = df_by_fuel_2019['Period'].dt.month

In [5]:
df_by_fuel_2019.head()

,Period,Region,Nuclear,Bituminous\ncoal,Anthracite\ncoal,Oil,LNG,Pumped-Storage,Fuel Cell,IGCC,Solar Power,Wind Power,Hydro Power,Marine Energy,Bio Energy,Waste Energy,Others*,Total,Num Days,Month
0,2019-12-01,Total,23250.0,36391.782,600.0,3771.040,39655.400,4700.0,464.146,346.33,10505.101538,1512.2214,1808.1034,255.5,899.651000,0.000000,1178.392,125337.667338,31,12
1,2019-11-01,Total,23250.0,36391.782,600.0,3874.893,39529.756,4700.0,446.211,346.33,9677.794676,1487.0004,1796.1334,255.5,887.576360,0.000000,1149.202,124392.178836,30,11
2,2019-10-01,Total,23250.0,36391.782,600.0,3874.893,39160.583,4700.0,446.211,346.33,9677.794676,1487.0004,1796.1334,255.5,887.576360,0.000000,1149.202,124023.005836,31,10
3,2019-09-01,Total,23250.0,36391.782,600.0,3968.616,38203.560,4700.0,425.971,346.33,9090.380216,1421.6754,1795.9844,255.5,891.400999,1067.711999,81.490,122490.402014,30,9
4,2019-08-01,Total,23250.0,36391.782,600.0,3968.616,38203.560,4700.0,425.971,346.33,9048.112311,1421.6754,1795.9844,255.5,891.400999,1592.911999,81.490,122973.334109,31,8


# Predictions for 2019

We couldn't find any real information broken down by hour, but we found hourly **predictions** for all fuels aggregated:
http://www.kpx.or.kr/www/contents.do?key=223&status=land&issueDate=20201122&__encrypted=M0x5Tkcr56VXs5Vrg0Td%2BPfmTOXKR3vojylqtBYvwmiWCM1iGvXybDUoUhIs2cBnx5hZKRoPP76ZY8Tr0uCWlFGvOk1PeCyOxwvEBWktkRk%3D
Unit: MW

In [6]:
df_predictions = pd.read_csv('bidforecastgen_land_2019.csv', sep="\t")
df_predictions = pd.melt(df_predictions, id_vars=['Date'], value_vars=['1h', '2h', '3h', '4h', '5h', '6h', '7h', '8h', '9h', '10h',
       '11h', '12h', '13h', '14h', '15h', '16h', '17h', '18h', '19h', '20h',
       '21h', '22h', '23h', '24h'],
        var_name='Hour', value_name='Total')

df_predictions['Hour'] = df_predictions['Hour'].replace({'h':''}, regex=True)
df_predictions['Hour'] = pd.to_numeric(df_predictions['Hour'])

df_predictions = df_predictions.sort_values(['Date', 'Hour']).reset_index(drop=True)

In [7]:
df_predictions['Date'] = pd.to_datetime(df_predictions['Date'], format='%Y%m%d').dt.date
df_predictions['Date'] = pd.to_datetime(df_predictions['Date'])


In [8]:
df_predictions['Nuclear'] = df_2019['Nuclear (MW)'].values[0]

# Assume Nuclear is constant every hour

df_predictions['Total non nuclear'] = df_predictions['Total'] - df_2019['Nuclear (MW)'].values[0]
df_predictions['Datetime'] = pd.to_datetime(df_predictions['Date']) + pd.to_timedelta(df_predictions.pop('Hour'), unit='H')

df_predictions = df_predictions.drop(columns=['Date'])
df_predictions.head()

,Total,Nuclear,Total non nuclear,Datetime
0,60140,23250,36890,2019-01-01 01:00:00
1,57700,23250,34450,2019-01-01 02:00:00
2,56170,23250,32920,2019-01-01 03:00:00
3,55200,23250,31950,2019-01-01 04:00:00
4,55200,23250,31950,2019-01-01 05:00:00


# HOUR

In order to estimate the hourly values per fuel, we do as follows:
1. We assume that nuclear production is constant every hour.
2. We initiate a table having one row per hour and day in 2019, and one colunmn for every fuel but nuclear, as we assume it to be constant. We set the values to be the averages given in df_by_fuel_2019.
3. We set solar power to zero at nighttime, i.e. from between 7pm and 6am.
4. We perform Iterative Proportion Fitting on this matrix to match values to hourly predictions.

In [9]:
df_by_hour = pd.DataFrame()

# fill dataframe with daily average by fuel

df_by_hour[['Datetime']] = df_predictions[['Datetime']].copy()
df_by_hour['Month'] = df_by_hour['Datetime'].dt.month

non_nuclear = ['Bituminous\ncoal', 'Anthracite\ncoal',
        'Oil', 'LNG', 'Pumped-Storage', 'Fuel Cell', 'IGCC', 'Solar Power',
        'Wind Power', 'Hydro Power', 'Marine Energy', 'Bio Energy',
        'Waste Energy', 'Others*']

non_nuclear_month = ['Bituminous\ncoal', 'Anthracite\ncoal',
        'Oil', 'LNG', 'Pumped-Storage', 'Fuel Cell', 'IGCC', 'Solar Power',
        'Wind Power', 'Hydro Power', 'Marine Energy', 'Bio Energy',
        'Waste Energy', 'Others*','Month']

df_by_hour = pd.merge(df_by_hour, df_by_fuel_2019[non_nuclear_month], on='Month', how='left')

In [10]:
df_by_hour['Solar Power'] = np.where((df_by_hour['Datetime'].dt.hour <=6) | (df_by_hour['Datetime'].dt.hour > 18), 
                                    0, df_by_hour['Solar Power'] )


In [11]:
df_by_hour = df_by_hour.drop(columns=['Month'])

df_by_hour_melted = pd.melt(df_by_hour, id_vars=['Datetime'], value_vars=non_nuclear,
        var_name='Fuel', value_name='total')

In [12]:
df_by_hour_melted.head()

,Datetime,Fuel,total
0,2019-01-01 01:00:00,Bituminous\ncoal,36391.777
1,2019-01-01 02:00:00,Bituminous\ncoal,36391.777
2,2019-01-01 03:00:00,Bituminous\ncoal,36391.777
3,2019-01-01 04:00:00,Bituminous\ncoal,36391.777
4,2019-01-01 05:00:00,Bituminous\ncoal,36391.777


In [13]:
xpj = df_by_hour_melted.groupby('Fuel')['total'].sum()
# replace values by hour by predictions
df_predictions = df_predictions.set_index('Datetime')

xip = df_predictions['Total non nuclear']

aggregates = [xip, xpj]
dimensions = [['Datetime'], ['Fuel']]

IPF = ipfn.ipfn(df_by_hour_melted, aggregates, dimensions)
    
df_by_hour_melted = IPF.iteration()

ipfn converged: convergence_rate not updating or below rate_tolerance


In [14]:
df_by_hour_melted.head()

,Fuel,Datetime,total
0,Bituminous\ncoal,2019-01-01 01:00:00,38297.560302
1,Bituminous\ncoal,2019-01-01 02:00:00,35764.460623
2,Bituminous\ncoal,2019-01-01 03:00:00,34176.082546
3,Bituminous\ncoal,2019-01-01 04:00:00,33169.071609
4,Bituminous\ncoal,2019-01-01 05:00:00,33169.071609


In [15]:
df_predictions = df_predictions.reset_index()
df_predictions_melted = df_predictions[['Datetime','Nuclear']].rename(columns={'Nuclear':'total'})
df_predictions_melted['Fuel'] = 'Nuclear'

df_by_hour_melted = df_by_hour_melted.append(df_predictions_melted)


In [16]:
# Reshape dataframe to have one column per fuel

df_by_hour_reshaped = df_by_hour_melted.pivot(index='Datetime', columns='Fuel', values='total')
df_by_hour_reshaped = df_by_hour_reshaped.reset_index()

df_by_hour_reshaped['timestamp'] = df_by_hour_reshaped['Datetime'].astype('int64') // 10**9
df_by_hour_reshaped['zone_name'] = "KR"

cols = df_by_hour_reshaped.columns.tolist()
cols = cols[:1] + cols[-2:] + cols[1:-2]

df_by_hour_reshaped = df_by_hour_reshaped[cols]

In [17]:
df_by_hour_reshaped.head()

Fuel,Datetime,timestamp,zone_name,Anthracite\ncoal,Bio Energy,Bituminous\ncoal,Fuel Cell,Hydro Power,IGCC,LNG,Marine Energy,Nuclear,Oil,Others*,Pumped-Storage,Solar Power,Waste Energy,Wind Power
0,2019-01-01 01:00:00,1546304400,KR,631.421108,569.798197,38297.560302,362.334712,1887.284922,364.466787,39829.775764,268.356600,23250.0,4538.051262,83.463687,4946.132012,0.0,1647.870261,1499.889185
1,2019-01-01 02:00:00,1546308000,KR,589.657283,532.110270,35764.460623,338.368957,1762.455016,340.360011,37195.331393,250.606800,23250.0,4237.892816,77.943183,4618.982049,0.0,1538.875861,1400.682635
2,2019-01-01 03:00:00,1546311600,KR,563.469311,508.478087,34176.082546,323.341250,1684.180527,325.243877,35543.405209,239.476803,23250.0,4049.678708,74.481556,4413.842934,0.0,1470.531011,1338.475250
3,2019-01-01 04:00:00,1546315200,KR,546.866479,493.495592,33169.071609,313.813880,1634.555523,315.660446,34496.105602,232.420530,23250.0,3930.353424,72.286929,4283.787416,0.0,1427.201270,1299.036581
4,2019-01-01 05:00:00,1546318800,KR,546.866479,493.495592,33169.071609,313.813880,1634.555523,315.660446,34496.105602,232.420530,23250.0,3930.353424,72.286929,4283.787416,0.0,1427.201270,1299.036581


In [18]:
df_by_hour_reshaped.to_csv('korea_hourly.csv', index=False)